<a href="https://colab.research.google.com/github/adechielie/ATP-Prediction/blob/main/climatedata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas


# ======================================================
# 🏙️ Téléchargement et préparation des données atp
# ======================================================

# URL de base du dépôt GitHub
base_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/"

# Générer la liste des fichiers à récupérer (de 2000 à 2024)
years = range(2000, 2025)
file_urls = [f"{base_url}atp_matches_{year}.csv" for year in years]

# Télécharger et fusionner les fichiers
dfs = []  # Liste pour stocker chaque DataFrame

for url in file_urls:
    try:
        df = pd.read_csv(url)
        dfs.append(df)
        print(f"✅ {url} téléchargé avec succès")
    except Exception as e:
        print(f"⚠️ Erreur lors du téléchargement de {url}: {e}")

# Fusionner tous les fichiers en un seul DataFrame
atp_data = pd.concat(dfs, ignore_index=True)

# Filtrer les lignes où 'tourney_name' commence par 'Davis Cup' et compter
count = atp_data[atp_data['tourney_name'].str.startswith('Davis Cup', na=False)].shape[0]

print(f"il y'a eu {count} matchs de Davis Cup entre 2000 et 2024")

# Nombre de matchs total du dataset
print("Il y'a", atp_data.shape[0],"de lignes dans le dataset")

# Supression des matchs de Davis Cup
atp_data = atp_data[-atp_data['tourney_name'].str.startswith("Davis Cup")]

# Nouveau nombre de matchs total du dataset
print("En enlevant les matchs de coupe Davis il y'a",atp_data.shape[0], "lignes dans le dataset")

# Création de la colonne Location

atp_data['Location']=atp_data['tourney_name']

# Modification des noms de tournois qui ne correpondent pas exacxtement aux noms des villes par les noms exactes des villes

atp_data['Location'].replace(to_replace={'Stuttgart Outdoor': 'Stuttgart',
                                                'Paris Masters': 'Paris',
                                                'Stuttgart Masters': 'Stuttgart',
                                                'Miami Masters': 'Miami',
                                                'Indian Wells Masters': 'Indian Wells',
                                                'Monte Carlo Masters': 'Roquebrune-Cap-Martin',
                                                'Hamburg Masters': 'Hamburg',
                                                'Rome Masters': 'Rome',
                                                'Canada Masters': 'Toronto',
                                                'Cincinnati Masters': 'Cincinnati',
                                                'Shanghai Masters': 'Shanghai',
                                                'Atlanta Masters': 'Atlanta',
                                                'Roland Garros': 'Paris',
                                                'Buenos Aires Masters': 'Buenos Aires',
                                                'Dubai Masters': 'Dubai',
                                                'Vienna Masters': 'Vienna',
                                                'Mexico City Masters': 'Mexico City',
                                                'Wimbledon': 'London',
                                                'Madrid Masters': 'Madrid',
                                                'Barcelona Masters': 'Barcelona',
                                                'US Open': 'New York',
                                                'London Olympics': 'London',
                                                'Paris Olympics': 'Paris',
                                                'Tokyo Olympics': 'Tokyo',
                                                'Rio Olympics': 'Rio de Janeiro',
                                                'Australian Open': 'Melbourne',
                                                'Laver Cup': 'London',
                                                'Cologne 2': 'Cologne',
                                                'Cologne 1': 'Cologne',
                                                'Us Open': 'New York',
                                                'Belgrade 2': 'Belgrade',
                                                'Belgrade 1': 'Belgrade',
                                                'Adelaide 2': 'Adelaide',
                                                'Adelaide 1': 'Adelaide',
                                                'ATP Rio de Janeiro': 'Rio de Janeiro',
                                                'Tour Finals': 'London',
                                                'Next Gen Finals': 'Milan',
                                                'Sydney Olympics': 'Sydney',
                                                'Beijing Olympics': 'Beijing',
                                                'Great Ocean Road Open': 'Melbourne',
                                                'Murray River Open': 'Melbourne',
                                                 }, inplace = True
                                         )


# Liste des villes après remplacement
locations = {
    "Stuttgart": None, "Paris": None, "Miami": None, "Indian Wells": None, "Roquebrune-Cap-Martin": None,
    "Hamburg": None, "Rome": None, "Toronto": None, "Cincinnati": None, "Shanghai": None, "Atlanta": None,
    "Buenos Aires": None, "Dubai": None, "Vienna": None, "Mexico City": None, "London": None, "Madrid": None,
    "Barcelona": None, "New York": None, "Tokyo": None, "Rio de Janeiro": None, "Melbourne": None, "Auckland": None,
    "St. Poelten": None, "Munich": None, "Gstaad": None, "Newport": None, "Bastad": None, "Amsterdam": None,
    "Kitzbuhel": None, "Palermo": None, "Toulouse": None, "Basel": None, "Bogota": None, "Casablanca": None,
    "Lyon": None, "Memphis": None, "Rotterdam": None, "Washington": None, "Indianapolis": None, "Los Angeles": None,
    "San Jose": None, "Stockholm": None, "Scottsdale": None, "Moscow": None, "Umag": None, "s Hertogenbosch": None,
    "Long Island": None, "Doha": None, "Estoril": None, "San Marino": None, "Copenhagen": None, "Marseille": None,
    "Delray Beach": None, "Halle": None, "Santiago": None, "Mallorca": None, "Dusseldorf": None, "Brighton": None,
    "Orlando": None, "Tashkent": None, "Nottingham": None, "Bucharest": None, "Chennai": None, "Milan": None,
    "Sopot": None, "Vina del Mar": None, "Costa Do Sauipe": None, "Houston": None, "Acapulco": None, "Amersfoort": None,
    "Bangkok": None, "Metz": None, "Valencia": None, "Beijing": None, "Athens": None, "New Haven": None,
    "Ho Chi Minh City": None, "Zagreb": None, "Poertschach": None, "Mumbai": None, "Las Vegas": None, "Warsaw": None,
    "Brisbane": None, "Johannesburg": None, "Belgrade": None, "Kuala Lumpur": None, "Eastbourne": None, "Nice": None,
    "Montpellier": None, "Winston-Salem": None, "Sao Paulo": None, "Shenzhen": None, "Quito": None, "Istanbul": None,
    "Geneva": None, "Sofia": None, "Marrakech": None, "Los Cabos": None, "Chengdu": None, "Antwerp": None,
    "Budapest": None, "Antalya": None, "Pune": None, "Cordoba": None, "Zhuhai": None, "Cologne": None, "Sardinia": None,
    "St. Petersburg": None, "Nur-Sultan": None, "Singapore": None, "Marbella": None, "Cagliari": None, "Parma": None,
    "San Diego": None, "Dallas": None, "Seoul": None, "Tel Aviv": None, "Astana": None, "Florence": None, "Gijon": None,
    "Naples": None, "Banja Luka": None, "Hangzhou": None, "Almaty": None
}



# ======================================================
# 🏙️ Fonction pour récupérer les coordonnées des villes sur l'API nominatim openstreet
# ======================================================

def get_all_city_coordinates():
    """Récupère les coordonnées GPS de toutes les villes en une seule fois."""
    headers = {
        "User-Agent": "prediction_atp (adechielie@yahoo.fr)"
    }

    for city in locations.keys():
        geocode_url = f"https://nominatim.openstreetmap.org/search?city={city}&format=json"
        response = requests.get(geocode_url, headers=headers)  # Ajout des headers
        time.sleep(1)  # Pause pour éviter le blocage

        try:
            response_json = response.json()
            if response_json:
                locations[city] = (float(response_json[0]["lat"]), float(response_json[0]["lon"]))
            else:
                print(f"⚠️ Coordonnées introuvables pour {city}")
        except requests.exceptions.JSONDecodeError:
            print(f"❌ Erreur JSON pour {city}")

get_all_city_coordinates()


"""Définitions de listes pour récupérer les températures et la vitesse du vent journalières pour toutes les villes en un seul appel API."""
latitudes = []
longitudes = []
city_names = []
print("📍 Locations avec coordonnées:", locations)

for city, coords in locations.items():
    if coords is not None:  # Vérifier que les coordonnées existent
        latitudes.append(str(coords[0]))
        longitudes.append(str(coords[1]))
        city_names.append(city)

print("📍 Latitudes:", latitudes)
print("📍 Longitudes:", longitudes)
print("📍 Cities:", city_names)



# ======================================================
# 🏙️ Récupération des données climatiques via l'API open-mete
# ======================================================
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Définition des dates pour limiter l'appel API
START_DATE = "2000-01-01"
END_DATE = "2024-01-31"

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": ",".join(latitudes),
  "longitude": ",".join(longitudes),
  "start_date": START_DATE,
  "end_date": END_DATE,
  "daily": "temperature_2m_max,wind_speed_10m_max",
  "timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)
print(responses)

# Liste pour stocker les DataFrames
dataframes = []

# Associer chaque réponse API à une ville
for city, response in zip(city_names, responses):
    try:
        print(f"Traitement de {city} ({response.Latitude()}°N, {response.Longitude()}°E)")

        # Vérifier si la réponse API est valide
        if response is None:
            print(f"❌ Erreur : Réponse API manquante pour {city}")
            continue  # Passer à la ville suivante

        # Extraction des données
        daily = response.Daily()

        # Vérifier si les données existent
        if daily is None:
            print(f"⚠️ Aucune donnée météo trouvée pour {city}, passage à la ville suivante.")
            continue

        # Récupération des variables (sécurisée car on a déjà validé leur présence)
        daily_temperature = daily.Variables(0).ValuesAsNumpy()
        daily_wind_speed = daily.Variables(1).ValuesAsNumpy()

        # Vérifier si les données sont valides
        if daily_temperature is None or daily_wind_speed is None:
            print(f"⚠️ Données vides pour {city}, passage à la ville suivante.")
            continue

        # Création du DataFrame
        daily_data = {
            "city": city,
            "date": pd.date_range(start=START_DATE, end=END_DATE, freq="D"),
            "temperature_2m_max": daily_temperature,
            "wind_speed_10m_max": daily_wind_speed,
        }

        df = pd.DataFrame(daily_data)
        print(f"✅ Traitement de {city} réussi")
        dataframes.append(df)

    except Exception as e:
        print(f"🚨 Erreur lors du traitement de {city} : {e}")
        continue  # Passer à la ville suivante sans stopper le programme

# Concaténer tous les DataFrames en un seul
if dataframes:
    climate_data = pd.concat(dataframes, ignore_index=True)
    print("✅ Concaténation réussie !")
    print(climate_data.head())
else:
    print("❌ Aucune donnée valide n'a été récupérée.")



✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2000.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2001.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2002.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2003.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2004.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2005.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2006.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2007.csv téléchargé avec succès
✅ https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2008.csv téléchargé avec succès
✅